**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

---

Mentoría #13 - Cómo hacer un Clasificador de Pliegos todoterreno (y de otros tipos de textos) usando NLP

---
Integrantes Grupo [1|2]:
*   Bruce Wayne: bruce.w@wayne-entreprises.com
*   Alfred Pennyworth: alfred@msn.com
*   Richard Grayson: robin_132@gmail.com
---

Edición 2023


# Introducción

En esta Mentoría, trabajaremos con un conjunto de datos que comprende aproximadamente 100.000 pliegos y licitaciones de diversos organismos nacionales, tanto públicos como privados. Estos datos se obtuvieron de un sistema/servicio diseñado para monitorear oportunidades de negocios, capturar la información en una base de datos, normalizarla y, posteriormente, clasificarla para informar a los usuarios según sus áreas de interés. Los usuarios de este sistema reciben alertas automáticas cada vez que se publica una oportunidad comercial que coincide con su perfil.

## Desafío

Como parte del proceso de clasificación, los pliegos se etiquetan utilizando principalmente reglas estáticas, como palabras clave, lo cual deja margen para optimizaciones. El título de cada pliego y principalmente la descripción de los objetos que se están licitando son campos de tipo texto y escritos por personas, por lo que naturalmente presentan ambigüedades y características propias del lenguaje que llevan a que un enfoque rígido y estático de clasificación, como el actual, resulte limitado y poco eficiente.

El Desafío es utilizar las técnicas de Aprendizaje Automático, logrando así un "Clasificador" que utilice técnicas de Procesamiento de Lenguaje Natural (NLP) para clasificar de manera más eficiente en qué rubro o categoría se encuentra un pliego, basándose en su texto descriptivo y otros campos relacionados.

## Interés General

Más allá de la aplicación específica en este conjunto de datos, este problema, al estar vinculado al Procesamiento de Lenguaje Natural (PLN) y la clasificación, tiene la ventaja de poder ser utilizado posteriormente para otros tipos de contenido. De esta manera, el clasificador desarrollado podría aplicarse para categorizar libros, noticias, textos, tweets, publicaciones, artículos, etc.

## Descripción del dataset

A continuación se enumeran las diferentes variables del dataset, así como una breve descripción de su significado:

- **id**: Clave única y primaria autoincremental de la tabla;
- **cargado**: Fecha de carga del pliego;
- **idexterno**: Id del pliego en la fuente;
- **referencia**: Campo auxiliar obtenido de la fuente;
- **objeto**: Campo principal, descripción del producto o servicio objeto de la licitación;
- **rubro**: Campo de categorización disponible en la fuente. No siempre está disponible;
- **agencia**: Empresa o Ente que lanza la licitación;
- **apertura**: Fecha de apertura del pliego (vencimiento para presentarse al pliego);
- **subrubro**: Subcategoría del pliego (también obtenido desde la fuente);
- **pais**: País donde se lanza la licitación;
- **observaciones**: Campo auxiliar donde se guardan datos extra que puede variar según la fuente;
- **monto**: Monto del pliego, no siempre está publicado;
- **divisaSimboloISO**: Moneda en la que se especifica el pliego;
- **visible**: campo binario que determina si el pliego va a ser visualizado por el sistema (True/False);
- **categoría**: Tipo de pliego, categorización entre diversos tipos de pliegos (Compra Directa, Licitación Simple, Subasta, etc.);
- **fuente**: Fuente de donde se obtuvo la licitación.

# Trabajo Práctico #2

Como vimos en el primer Trabajo Práctico, en esta segunda entrega tenemos como objetivo preparar los datos para poder alimentar un modelo de predicción. Aplicaremos diferentes tipos de prácticas y estrategias para preparar el dataset y resolver todas las cuestiones detectadas en el TP1

A continuación se enumeran los ejercicios propuestos como guía:

## Ejercicio 1: Limpieza de Datos

Según lo vimos en el TP1, aplicar las técnicas propuesta para limpiar los datos. En todos los casos, además de aplicar la técnica, redactar brevemente la explicación del criterio tomado.

1.   Selección de Variables. Como vimos, hay variables que pueden descartarse.
2.   Corrección de los diferentes Tipos de Variables. Aplicar las conversiones de tipo necesarias.
3.   Selección de Registros. Filtrar las filas que no utilizaremos para la clasificación.
4.   Curación de la variable [monto]. Tip: usar la variable [divisaSimboloISO]
5.   Curación de la variable [fecha]. Incluye la eliminación de Outliers.
6.   En [agencia], normalizar si es que existen entradas con valores equivalentes.
7.   (Opcional) Para las variables categóricas aplicar técnicas de encoding, de manera de ir transformando variables de texto en números que los algoritmos puedan procesar.

## Ejercicio 2: Variable [rubro]

1.   Unificar categorías (rubros) equivalentes entre distintas fuentes. Usar un criterio definido por el grupo y explicarlo en pocas palabras.
2.   Utilizar visualizaciones que permitan ver todas las categorías y verificar que sean valores válidos.
3.   En caso de haber registros con categorías inválidas, tomar algún criterio para imputar la categoría o eliminar el registro. Desarrollar la decisión tomada.
4.   Resolver si se va a utilizar subrubro para aperturar la categorización en algún caso. Definir luego qué postura tomar con respecto a la variable [rubro]
5.   En las casos en que existe más de una categoría, tomar alguna postura para que los registros finalmente queden con una única etiqueta.

## Ejercicio 3: Variable [objeto]

1.   Aplicar técnicas de lematización para eliminar artículos, conectores, stop_words en general. Quitar acentos (tildes), mayúsculas, plurales, caractéres inválidos, abreviaturas, etc., dejando solo palabras importantes para la clasificación.
2.   Utilizar técnicas de visualización para detectar otros términos muy frecuentes que no aportan valor a la clasificación.
3.   (Opcional) Aplicar técnicas de tokenización para convertir la variable objeto en un array de tokens.

## Ejercicio 4: Exportación

Con el dataset ya transformado, generar una versión final y exportarla a un nuevo archivo. Este es el resultado final del TP2.

## Ejercicio 5: Análisis

En base a todo lo aprendido en las materias obligatorias cursadas, qué tipo de modelos de clasificación piensan que podrían aplicarse al dataset obtenido? Desarrollar brevemente una justificación en comparación con otros modelos posibles.

# Desarrollo

In [1]:
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np

pd.set_option('display.max_rows', None)

df = pd.read_csv("./datos/Mentoria_Dataset_0.csv" , encoding='latin-1')

In [2]:
#en el practico 1 decidimos tirar los datos que tienen categoría 2:
#tiro los que tienen categoria 2
df_fil = df[df['categoria']==1]

#variables que decidimos relevantes el práctico pasado: 
variables = ['referencia', 'objeto', 'rubro','agencia','apertura','monto','fuente']

#filtro el dataframe con estas variables:
df_fil = df_fil[variables]

In [3]:
#Hacemos el filtrado de fechas hecho en el práctico anterior:

today_date = pd.Timestamp.today().date()
min_date = pd.to_datetime('13/12/2015')
df_fil.reset_index(inplace=True)  #reseteo indices, es util cuando indice=nro de fila

#fechas malas que encontramos:
fecha_mala1 = df_fil['apertura'][62]
fecha_mala2 = df_fil['apertura'][2025]
fecha_mala3 = df_fil['apertura'][11907]

#removemos las fechas malas
df_fil = df_fil.drop(df_fil[df_fil['apertura'] == fecha_mala1].index)
df_fil = df_fil.drop(df_fil[df_fil['apertura'] == fecha_mala2].index)
df_fil = df_fil.drop(df_fil[df_fil['apertura'] == fecha_mala3].index)

#convierto a formato datetime
df_fil['apertura'] = pd.to_datetime(df_fil['apertura'],format='%d/%m/%Y %H:%M')
df_fil['apertura'] = df_fil['apertura'].dt.tz_localize(None)
df_fil['apertura'] = df_fil['apertura'].dt.date

#removemos outliers
df_fil = df_fil.drop(df_fil[df_fil['apertura'] > today_date].index)
df_fil = df_fil.drop(df_fil[pd.to_datetime(df_fil['apertura']) < min_date].index)


/tmp/ipykernel_10294/218546107.py:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  min_date = pd.to_datetime('13/12/2015')


In [4]:
#precio del dolar oficial
dolar_value = 255

df_fil['monto'] = df_fil['monto'].replace(r'\N', np.nan)
df_fil['monto'] = df_fil['monto'].replace('0', 0)

#remuevo las strings de espacios, pesos y dolares donde las haya. Cambio la representacion de string a float
j=0
for monto, i in zip(df_fil['monto'],df_fil.index):
    if " " in str(monto):
        df_fil['monto'][i] = str(df_fil['monto'][i]).replace(' ','')
    if "$" in str(monto):
        df_fil['monto'][i] = str(df_fil['monto'][i]).replace('$','')
    if "." in str(monto):
        df_fil['monto'][i] = str(df_fil['monto'][i]).replace('.','')
    if "," in str(monto):
        df_fil['monto'][i] = str(df_fil['monto'][i]).replace(',','.')    
    if "U$S" in str(monto):
        df_fil['monto'][i] = str(df_fil['monto'][i]).replace('US','')
        #convierto a pesos y reemplazo la entrada
        df_fil['monto'][i] = float(df_fil['monto'][i]) * dolar_value  
        j+=1
    df_fil['monto'][i] = float(df_fil['monto'][i]) 
        

/tmp/ipykernel_10294/3566752287.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fil['monto'][i] = float(df_fil['monto'][i])
/tmp/ipykernel_10294/3566752287.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fil['monto'][i] = str(df_fil['monto'][i]).replace(' ','')
/tmp/ipykernel_10294/3566752287.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fil['monto'][i] = str(df_fil['monto'][i]).replace('$','')
/tmp/ipykernel_

Empiezo a hacer el ejercicio 3:

a- Aplicar técnicas de lematización para eliminar artículos, conectores, stop_words en general. Quitar acentos (tildes), mayúsculas, plurales, caractéres inválidos, abreviaturas, etc., dejando solo palabras importantes para la clasificación.

b- Utilizar técnicas de visualización para detectar otros términos muy frecuentes que no aportan valor a la clasificación.
    
c- (Opcional) Aplicar técnicas de tokenización para convertir la variable objeto en un array de tokens.


donde aplicar la limpieza:
* objeto
* rubro
* agencia

In [21]:
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

table = str.maketrans(string.punctuation,' '*len(string.punctuation)) #tabla para eliminar signos de puntuacion


#funcion para lematizar una palabra
def lemmatize_word(word):
    stemmer = SnowballStemmer('spanish')
    return stemmer.stem(word)

#defino los datasets a limpiar
df_test = df_fil[['objeto','rubro']]

df_test.reset_index(inplace=True)

i=0
for i in range (len(df_test)):    
    df_test['objeto'][i] = str(df_test['objeto'][i]).lower()
    df_test['objeto'][i] = str(df_test['objeto'][i]).translate(table) #elimino signos de puntuacion
    sent = str(df_test['objeto'][i])   # variable a analizar
    if "á" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('á','a')
    if "é" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('é','e')
    if "í" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('í','i')
    if "ó" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('ó','o')
    if "ú" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('ú','u')
    i+=1

print(df_test['objeto'][0:100])
    


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/santiago/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/santiago/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/tmp/ipykernel_10294/2929354565.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['objeto'][i] = str(df_test['objeto'][i]).lower()
/tmp/ipykernel_10294/2929354565.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['objeto'][i] = str(df_test['objeto'][i]).translate(table) #elimino signos de puntuacion
/tmp/ipykernel_10294/2929354565.py:

0                     adquisicion de bolsas de alimento
1     subasta de 105 cabezas de ganado vacuno  3 lot...
2     subasta de 455 cabezas de ganado vacuno  10 lo...
3     subasta de 485 cabezas de ganado vacuno  7 lot...
4     subasta de 280 cabezas de ganado vacuno  4 lot...
5     subasta de 105 cabezas de ganado vacuno  3 lot...
6     subasta de 245 cabezas de ganado vacuno  4 lot...
7     adquisicion de repuestos e insumos para equipa...
8     estudio sobre costos de produccion y precios d...
9     obras de  construccion muelle de ushuaia y mue...
10    obra mejoramiento del sistema de riego de anju...
11     obra mejoramiento del sistema hidrico de angulos
12                                                  nan
13             adquisicion de minicargador y accesorios
14                     subasta publica de ganado equino
15                               venta de ganado equino
16                          alimento  animales bioterio
17    adquisicion de insumos para comedor   rubr

In [25]:
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

nltk.download('punkt')
nltk.download('stopwords') #creo que estamos haciendo lo mismo que al eliminar las preposiciones con esto
nltk.download('wordnet')

df_test0 = df_test

print('Tokenizamos las oraciones en palabras')
print()
df_test0['objeto'] = df_test0['objeto'].apply(word_tokenize)
print(df_test0['objeto'][:10])
print('*'*100)
print('Removemos stopwords (son como las contracciones?)')
print()
stop_words = set(stopwords.words('spanish'))
df_test0['objeto'] = df_test0['objeto'].apply(lambda words: [word for word in words if word not in stop_words])
print(df_test0['objeto'][:10])
print('*'*100)
print('Aplicamos lematización a cada palabra')
print()
df_test0['objeto'] = df_test0['objeto'].apply(lambda words: [lemmatize_word(word) for word in words])
print(df_test0['objeto'][:10])
print('*'*100)





# corpus = []
# text_data = ""
# for i in range(0, len(df_test)):
#     text_data = str(df_test['objeto'][i])
#     text_data = text_data.split()
#     wl = WordNetLemmatizer()
#     text_data = [wl.lemmatize(word) for word in text_data if not word in set(stopwords.words('spanish'))]
#     text_data = ' '.join(text_data)
#     corpus.append(text_data)
    


[nltk_data] Downloading package punkt to /home/santiago/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/santiago/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/santiago/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Tokenizamos las oraciones en palabras



TypeError: expected string or bytes-like object

In [18]:
print(corpus[0:100])

['adquisicion bolsas alimento', 'subasta 105 cabezas ganado vacuno (3 lotes 35 novillos gordos), establecimiento grl paz, ruta provincial nro 6 km 153,5, localidad ordoñez, provincia cordoba', 'subasta 455 cabezas ganado vacuno (10 lotes 35 novillos gordos c/u 3 lotes 35 vacas gordas c/u), establecimiento general urquiza, ruta provincial nro 6, sitio localidad arroyo cle, provincia rio', 'subasta 485 cabezas ganado vacuno (7 lotes 35 vacas invernada c/u 2 lotes 120 vaquillonas 2 3 años invernada c/u), establecimiento general avalos, 1ra seccion rural km 173, monte caseros, provincia corrientes', 'subasta 280 cabezas ganado vacuno (4 lotes 35 novillos gordos c/u, 2 lotes de55 vaquillonas 1 2 años invernada c/u, 1 lote 4 toros refugo, 1 lote 20 vacas conserva 1 lote 6 novillos refugo), establecimiento coronel pringles, s', 'subasta 105 cabezas ganado vacuno (3 lotes 35 novillos gordos), establecimiento general paz, ruta provincial nro 6 km 153,5, localidad ordoñez, provincia cordoba.', '